In [ ]:
SELECT 
    p.display_name,
    p.id ,
    COALESCE(bp.total_count, pc.total_count, 0) AS population_count,
    CASE 
        WHEN bp.total_count IS NOT NULL THEN 'block_pops'
        WHEN pc.total_count IS NOT NULL THEN 'population_census'
        ELSE 'none'
    END AS data_source
FROM 
    population p
LEFT JOIN (
    SELECT 
        bp.population_id,
        SUM(bp.count) AS total_count
    FROM 
        block_pops bp
    JOIN 
        block_territory bt ON bp.bt_id = bt.id
    JOIN 
        blocks b ON bt.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        bp.population_id
) bp ON bp.population_id = p.id
LEFT JOIN (
    SELECT 
        pc.population_id,
        SUM(pc.population_count) AS total_count
    FROM 
        population_census pc
    JOIN 
        territory_joins tj ON pc.territory_id = tj.id
    JOIN 
        blocks b ON tj.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        pc.population_id
) pc ON pc.population_id = p.id
where p.id in (2,3)


In [ ]:
SELECT 
    l.livestock_name,
    l.id AS livestock_id,
    l.coefficient AS coefficient,
    COALESCE(bl.total_count, lc.total_count, 0) AS livestock_count,
    CASE 
        WHEN bl.total_count IS NOT NULL THEN 'block_livestocks'
        WHEN lc.total_count IS NOT NULL THEN 'livestock_census'
        ELSE 'none'
    END AS data_source
FROM 
    livestocks l
LEFT JOIN (
    SELECT 
        bl.livestock_id,
        SUM(bl.count) AS total_count
    FROM 
        block_livestocks bl
    JOIN 
        block_territory bt ON bl.bt_id = bt.id
    JOIN 
        blocks b ON bt.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        bl.livestock_id
) bl ON bl.livestock_id = l.id
LEFT JOIN (
    SELECT 
        lc.livestock_id,
        SUM(lc.livestock_count) AS total_count
    FROM 
        livestock_census lc
    JOIN 
        territory_joins tj ON lc.tj_id = tj.id
    JOIN 
        blocks b ON tj.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        lc.livestock_id
) lc ON lc.livestock_id = l.id
ORDER BY 
    l.livestock_name;

In [ ]:
SELECT 
    c.crop_name,
    c.id as crop_id,
    c.coefficient,
    COALESCE(bc.total_area, cc.total_area, 0) AS crop_area,
    CASE 
        WHEN bc.total_area IS NOT NULL THEN 'block_crops'
        WHEN cc.total_area IS NOT NULL THEN 'crop_census'
        ELSE 'none'
    END AS data_source
FROM 
    crops c
LEFT JOIN (
    SELECT 
        bc.crop_id,
        SUM(bc.area) AS total_area
    FROM 
        block_crops bc 
    JOIN 
        block_territory bt ON bc.bt_id = bt.id
    JOIN 
        blocks b ON bt.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        bc.crop_id
) bc ON bc.crop_id = c.id
LEFT JOIN (
    SELECT 
        cc.crop_id,
        SUM(cc.crop_area) AS total_area
    FROM 
        crop_census cc
    JOIN 
        territory_joins tj ON cc.territory_id = tj.id
    JOIN 
        blocks b ON tj.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        cc.crop_id
) cc ON cc.crop_id = c.id


In [ ]:
SELECT 
    wt.waterbody_name,
    wt.id as waterbody_id,
    COALESCE(bw.total_storage, wc.total_storage, 0) AS storage_capacity,
    COALESCE(bw.total_count, wc.total_count, 0) AS waterbody_count
FROM 
    waterbody_types wt
LEFT JOIN (
    SELECT 
        bw.wb_type_id as waterbody_id,
        sum(bw.storage) as total_storage,
        SUM(bw.count) AS total_count
    FROM 
        block_waterbodies bw 
    JOIN 
        block_territory bt ON bw.bt_id = bt.id
    JOIN 
        blocks b ON bt.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        bw.wb_type_id
) bw ON bw.waterbody_id = wt.id
LEFT JOIN (
    SELECT 
        wc.waterbody_id,
        SUM(wc.storage_capacity) AS total_storage,
        COUNT(wc.waterbody_id) as total_count
    FROM 
        waterbodies_census wc 
    JOIN 
        territory_joins tj ON wc.tj_id = tj.id
    JOIN 
        blocks b ON tj.block_id = b.id
    WHERE 
        b.lgd_code = 4876
    GROUP BY 
        wc.waterbody_id
) wc ON wc.waterbody_id = wt.id


In [ ]:
SELECT 
    ge.extractable,
    coalesce(bg.extraction,ge.extraction) as extraction,
    ge.stage_of_extraction,
    ge.category,
    ge.block_id AS block_id
FROM 
    groundwater_extractions ge
JOIN 
    blocks b ON b.id = ge.block_id
join 
	block_territory bt on bt.block_id = ge.block_id 
left join(
	select 
		bg.extraction,
		b.lgd_code 
	from 
		block_groundwater bg 
	join
		block_territory bt on bt.id = bg.bt_id
	join 
		blocks b on b.id = bt.block_id
	where 
		b.lgd_code = 4876
) bg on bg.lgd_code = b.lgd_code 
where 
	b.lgd_code = 4876


In [ ]:
SELECT 
    l.catchment,
    COALESCE(sum(bl.catchment_area), sum(lc.catchment_area), 0) AS catchment_area
FROM 
    lulc l
LEFT JOIN (
SELECT 
    SUM(bl.area) AS catchment_area,
    bl.lulc_id 
FROM 
     block_lulc bl 
JOIN 
    block_territory bt on bt.id = bl.bt_id 
JOIN 
    blocks b ON b.id = bt.block_id
WHERE 
    b.lgd_code = 4876 
group by 
	bl.lulc_id  
) bl ON bl.lulc_id = l.id
LEFT JOIN (
SELECT 
    SUM(lc.lulc_area) AS catchment_area,
    lc.lulc_id 
FROM 
    lulc_census lc 
JOIN 
    territory_joins tj ON tj.id = lc.tj_id
JOIN 
    blocks b ON b.id = tj.block_id
WHERE 
    b.lgd_code = 4876 
group by 
	lc.lulc_id 
) lc ON lc.lulc_id = l.id
group by 
	l.catchment 

In [ ]:
SELECT 
    TO_CHAR(r.observation_date , 'FMMon-YY') AS month_year,
    coalesce(br.actual,rd.actual,0) as actual,
    coalesce(br.normal,rd.normal,0) as normal
FROM 
    rainfall_data r
LEFT JOIN (
SELECT 
    TO_CHAR(br.month_year , 'FMMon-YY') AS date,
    ROUND(SUM(br.actual)::NUMERIC, 2) AS actual,
    ROUND(SUM(br.normal)::NUMERIC, 2) AS normal
FROM 
    block_rainfall br 
where 
	br.bt_id = 5
GROUP BY 
    TO_CHAR(br.month_year, 'FMMon-YY')
) br ON br.date = TO_CHAR(r.observation_date , 'FMMon-YY')
LEFT JOIN (
SELECT 
    TO_CHAR(rd.observation_date, 'FMMon-YY') AS date,
    ROUND(SUM(rd.actual)::NUMERIC, 2) AS actual,
    ROUND(SUM(rd.normal)::NUMERIC, 2) AS normal
FROM 
    rainfall_data rd 
where 
	rd.district_id = 745
GROUP BY 
    TO_CHAR(rd.observation_date, 'FMMon-YY')
) rd ON rd.date = TO_CHAR(r.observation_date , 'FMMon-YY')
group by 
	TO_CHAR(r.observation_date , 'FMMon-YY'),br.actual,br.normal,rd.normal,rd.actual
order by
	min(r.observation_date)